## Othello Funcs

In [ ]:
GRID_SIZE = 8

def is_valid_move(x, y,board,current_player):
    if board[y][x] != 0:
        return False
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board,current_player):
            return True
    return False

def check_direction(x, y, dx, dy,board,current_player):
    opponent = 1 if current_player == 2 else 2
    x, y = x + dx, y + dy
    if not (0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE) or board[y][x] != opponent:
        return False
    while 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE:
        if board[y][x] == 0:
            return False
        if board[y][x] == current_player:
            return True
        x, y = x + dx, y + dy
    return False

def make_move(x, y,board,current_player):
    board[y][x] = current_player
    directions = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for dx, dy in directions:
        if check_direction(x, y, dx, dy,board,current_player):
            flip_direction(x, y, dx, dy,board,current_player)

def flip_direction(x, y, dx, dy,board,current_player):
    x, y = x + dx, y + dy
    while board[y][x] != current_player:
        board[y][x] = current_player
        x, y = x + dx, y + dy

In [ ]:
def decide_winner(board,current_player):
    black = 0
    white = 0
    for x in board:
        for y in x:
            black += (y == 2)
            white += (y == 1)
    if(black + white != 64) and current_player == 2:
        return 0
    if(black + white != 64) and current_player == 1:
        return 1
    return int(black >= 32)

## Player class

In [ ]:
class Player():
    def __init__(self,**kwargs):
        pass

    def get_move(self,board,current_player):
        pass

### MCTS PLAYER STUFF

In [ ]:
import random 

def get_move_random(board,current_player):
    valid_moves = []
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if is_valid_move(x,y,board,current_player): valid_moves.append((x,y))
    if(len(valid_moves) == 0): return None    
    return valid_moves[random.randint(0,len(valid_moves)-1)]

In [ ]:
import math
from copy import deepcopy

class Node():
    def __init__(self,x,y,player_who_decides_next):
        self.x = x
        self.y = y
        self.children = []
        self.visits = 0
        self.wins = 0
        self.player = player_who_decides_next
    
    def generate_children(self,board):
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if is_valid_move(x,y,board,self.player): 
                    if self.player == 1: self.children.append(Node(x,y,2))
                    if self.player == 2: self.children.append(Node(x,y,1))
        if len(self.children) == 0:
            self.children = None

    def select(self,C):
        if(self.player == 2):
            ret = max(self.children,key = lambda child: child.wins/(child.visits+0.001) +  C*( 
            (2.0*math.log(self.visits))/(child.visits+0.001))**0.5)
            return ret
        else:
            ret = min(self.children,key = lambda child: child.wins/(child.visits+0.001) +  C*( 
            (2.0*math.log(self.visits))/(child.visits+0.001))**0.5)
            return ret

    def rollout(self,depth,C,board): # 1 == simulate from children
        self.visits += 1

        if depth == 0:
            ret = self.simulate(board)
            self.wins += ret
            return ret
        
        
        if self.children == None: 
            ret = decide_winner(board,self.player)
            self.wins += ret
            return ret
        if len(self.children) == 0: self.generate_children(board)
        if self.children == None: 
            ret = decide_winner(board,self.player)
            self.wins += ret
            return ret
        
        
        child = self.select(C)
        make_move(child.x,child.y,board,self.player)
        ret = child.rollout(depth-1,C,board)
        self.wins += ret
        return ret
    
    def simulate(self,board):
        current_player = self.player
        while True:
            move = get_move_random(board,current_player)
            if(move == None): break
            make_move(move[0],move[1],board,current_player)
            if current_player == 2: current_player = 1
            else: current_player = 2
        return decide_winner(board,current_player)
    
    def give_best_move(self,currentplayer):
        if currentplayer == 2 :child = max(self.children,key = lambda child: child.wins/(child.visits+0.001))
        else: child = min(self.children,key = lambda child: child.wins/(child.visits+0.001))
        return child.x,child.y

class MCTSPlayer(Player):
    def __init__(self,explorationFactor = 1,rollouts = 100,selectionDepth = 1):
        super().__init__()
        self.explorationFactor = explorationFactor
        self.rollouts = rollouts
        self.selectionDepth = selectionDepth

    def get_move(self,board,current_player):
        root = Node(-1,-1,current_player)

        for i in range(self.rollouts):
            sim = deepcopy(board)
            root.rollout(self.selectionDepth,self.explorationFactor,sim)

        return root.give_best_move(current_player)

### ALPHA-BETA PLAYER

In [ ]:
class AlphaBetaPlayer(Player):
    
    def __init__(self,depth = 2):
        super().__init__()
        self.depth = depth
    
    def naiveValue(self,board):
        val = 0
        for x in board:
            for y in x:
                if y == 2: val += 1
                if y == 1: val -= 1
        return val

    def minimax(self,board,depth,alpha,beta,maximizingPlayer,current_player):
    
        valid_moves = []
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if is_valid_move(x,y,board,current_player): valid_moves.append((x,y))

        if depth == 0: 
            # with torch.no_grad():
            #     val = OthelloNet(torch.tensor(FLATTEN(board),dtype=torch.float32,device = device))
            # return val
            return self.naiveValue(board)

        if len(valid_moves) == 0: return decide_winner(board,current_player) * 64

        if maximizingPlayer:
            value = -100
            for x,y in valid_moves:
            
                child = deepcopy(board)
                make_move(x,y,child,2)

                value = max(value,self.minimax(child,depth-1,alpha,beta,False,1))
                if value > beta:
                    break
                alpha = max(alpha,value)
            return value
        else:
            value = 100
            for x,y in valid_moves:

                child = deepcopy(board)
                make_move(x,y,child,1)

                value = min(value,self.minimax(child,depth-1,alpha,beta,True,2))
                if value < alpha:
                    break
                beta = min(beta,value)
            return value

    def get_move(self,board,current_player):
        if(current_player == 2):
            valid_moves = []
            for x in range(GRID_SIZE):
                for y in range(GRID_SIZE):
                    if is_valid_move(x,y,board,2): valid_moves.append((x,y))
    
            points = []
            for x,y in valid_moves:
            
                sim = deepcopy(board)
                make_move(x,y,sim,2)
    
                points.append(self.minimax(sim,self.depth,-1000,+1000,False,1))
    
            maxi = 0
            for i in range(len(valid_moves)):
                if points[i] > points[maxi]:
                    maxi = i
    
            return valid_moves[maxi]
        else:
            valid_moves = []
            for x in range(GRID_SIZE):
                for y in range(GRID_SIZE):
                    if is_valid_move(x,y,board,1): valid_moves.append((x,y))
    
            points = []
            for x,y in valid_moves:
            
                sim = deepcopy(board)
                make_move(x,y,sim,1)
    
                points.append(self.minimax(sim,self.depth,-1000,+1000,True,2))
    
            mini = 0
            for i in range(len(valid_moves)):
                if points[i] < points[mini]:
                    mini = i
    
            return valid_moves[mini]
            

### Random Player

In [ ]:
class RandomPlayer(Player):
    def __init__(self):
        super().__init__()

    def get_move(self, board, current_player):
        valid_moves = []
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if is_valid_move(x,y,board,current_player): valid_moves.append((x,y))
        return valid_moves[random.randint(0,len(valid_moves)-1)]